In [1]:
import os
import time

import torch
import torch.nn as nn

import numpy as np


from itkwidgets import view

import Kyle_model as kmdl
import dataset as dtst

In [7]:
model = kmdl.UNet().cuda()
criterion = nn.BCEWithLogitsLoss(pos_weight = torch.tensor([62.9]).cuda())
optimizer = torch.optim.Adadelta(model.parameters(),
                                 lr=0.003)

In [3]:
model.load_state_dict(torch.load("overfitted_model.tar"))

<All keys matched successfully>

In [3]:
dataset = dtst.UnetDataset(
    root_dir="/home/sci/kyle.anderson/lymph_nodes/Dataset",
    patch_size=64,
    min_probability=0.1
)
train_loader = torch.utils.data.DataLoader(
    dataset=dataset,
    batch_size=1,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)

In [4]:
single_sample = dataset[85]

In [5]:
single_sample["img"].shape

torch.Size([1, 64, 64, 64])

In [ ]:
view(single_sample["img"].squeeze(), single_sample["mask"].squeeze())

In [8]:
epoch = 0
while True:
    optimizer.zero_grad()
    vols = single_sample["img"].unsqueeze(0).cuda(non_blocking=True).type(torch.cuda.FloatTensor)
    masks = single_sample["mask"].unsqueeze(0).cuda(non_blocking=True).type(torch.cuda.FloatTensor)

    preds = model(vols)
    loss = criterion(preds, masks)

    loss.backward()
    optimizer.step()

    epoch_loss = loss.item()

    print(f"Epoch {epoch+1}: train loss: {epoch_loss:.4f}\r")
    time.sleep(0)
    if epoch % 50 == 0:
        torch.save(model.state_dict(), "overfitted_model.tar")
    epoch += 1


Epoch 1: train loss: 1.3915
Epoch 2: train loss: 1.3862
Epoch 3: train loss: 1.3811
Epoch 4: train loss: 1.3760
Epoch 5: train loss: 1.3712
Epoch 6: train loss: 1.3665
Epoch 7: train loss: 1.3621
Epoch 8: train loss: 1.3579
Epoch 9: train loss: 1.3539
Epoch 10: train loss: 1.3500
Epoch 11: train loss: 1.3464
Epoch 12: train loss: 1.3429
Epoch 13: train loss: 1.3395
Epoch 14: train loss: 1.3363
Epoch 15: train loss: 1.3332
Epoch 16: train loss: 1.3303
Epoch 17: train loss: 1.3274
Epoch 18: train loss: 1.3247
Epoch 19: train loss: 1.3221
Epoch 20: train loss: 1.3195
Epoch 21: train loss: 1.3171
Epoch 22: train loss: 1.3147
Epoch 23: train loss: 1.3124
Epoch 24: train loss: 1.3103
Epoch 25: train loss: 1.3082
Epoch 26: train loss: 1.3061
Epoch 27: train loss: 1.3042
Epoch 28: train loss: 1.3023
Epoch 29: train loss: 1.3005
Epoch 30: train loss: 1.2988
Epoch 31: train loss: 1.2971
Epoch 32: train loss: 1.2955
Epoch 33: train loss: 1.2939
Epoch 34: train loss: 1.2924
Epoch 35: train loss: 1

KeyboardInterrupt: 

In [13]:
num_positive = np.count_nonzero(single_sample["mask"])
d1, d2, d3 = single_sample["mask"].squeeze().shape
ratio = ((d1 * d2 * d3) - num_positive) / num_positive
print(f"{ratio = }")

ratio = 62.90638712823013


In [9]:
model.eval()
prediction = model(single_sample["img"].unsqueeze(0).cuda().type(torch.cuda.FloatTensor))

In [10]:
view(prediction.squeeze().detach().cpu(), single_sample["mask"].squeeze())

Viewer(geometries=[], gradient_opacity=0.22, interpolation=False, point_sets=[], rendered_image=<itk.itkImageP…